<a href="https://colab.research.google.com/github/MarohBJoshua22/Undergrad-Project/blob/main/BERT_%26_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch scikit-learn pandas

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 # Load the dataset from a CSV file
df = pd.read_csv('/content/drive/MyDrive/profanity_en.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/profanity_en.csv')
df_filtered = df[df['category_1'] == 'sexual anatomy / sexual acts']
df_filtered = df_filtered[['text']]
df_filtered['label'] = 1  # Binary label for grooming

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Non-grooming_set_utf8.csv')
df2 = df2[['text', 'label']]  # Assume 'label' column exists

In [ ]:
# Combine the two datasets
combined_df = pd.concat([df_filtered, df2], ignore_index=True)

# Convert 'label' column to numeric, handling non-numeric values
combined_df['label'] = pd.to_numeric(combined_df['label'], errors='coerce').fillna(0).astype(int)
# 'coerce' replaces non-numeric values with NaN, which are then filled with 0 and converted to integers.

In [ ]:
combined_df.shape

(1188, 2)

In [ ]:
combined_df.head(1000)

,text,label
0,69,1
1,@55,1
2,@ssfcker,1
3,@ssfucker,1
4,@ssfvcker,1
...,...,...
995,MRW,0
996,IDC,0
997,ILY,0
998,ADIH,0


In [ ]:
# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    combined_df['text'], combined_df['label'], test_size=0.2, random_state=42
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Function to encode text using BERT
def encode_texts(texts):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encodings)
    # Use the [CLS] token embedding for classification
    return outputs.last_hidden_state[:, 0, :].numpy()

In [ ]:
# Encode the training and test texts
train_embeddings = encode_texts(train_texts)
test_embeddings = encode_texts(test_texts)

In [ ]:
# Train a Logistic Regression model on the BERT embeddings
log_reg_model = LogisticRegression(max_iter=1000)
log_reg_model.fit(train_embeddings, train_labels)

LogisticRegression(max_iter=1000)

In [ ]:
# Make predictions on the test set
test_predictions = log_reg_model.predict(test_embeddings)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Accuracy: {accuracy}")
print(classification_report(test_labels, test_predictions, target_names=['non-grooming', 'grooming']))

Accuracy: 0.9621848739495799
              precision    recall  f1-score   support

non-grooming       0.94      0.81      0.87        36
    grooming       0.97      0.99      0.98       202

    accuracy                           0.96       238
   macro avg       0.95      0.90      0.92       238
weighted avg       0.96      0.96      0.96       238

